In [436]:
from scripts.path_finders import get_context_shp_path, get_data_dir, get_division_path, get_indicators_path
import pandas as pd
import shapefile
from simpledbf import Dbf5
from scripts.geo_utils import iter_shp_as_shapely, get_shapely_shape
from scripts.land_use import find_containing_radios, conn_postgis_db
import pandas as pd
from simpledbf import Dbf5
import re
from functools import partial

In [437]:
rus_shp_path = get_context_shp_path("rus-ciudad-2010-2011")
rus_csv_path = os.path.join(get_data_dir(), "uso-suelo-2011.csv")
clasif_path = os.path.join(get_data_dir(), "RUS_clasif_rama1.xlsx")
parcelas_csv = os.path.join(get_data_dir(), "parcelas-100112-wgs84.csv")

In [438]:
df = pd.read_csv(rus_csv_path, encoding="utf-8", sep=";")
df_clasif = pd.read_excel(clasif_path)

In [439]:
df[df.smp.str.startswith("041-023")]

,x,y,id,seccion,manzana,parcela,sm,smp,unificacio,parcelas_u,...,d41,d31,d21,barrio,comuna,zona,ano,rama1,sub_rama1,ss_rama1
47,-58.4712,-34.5643,562227,41,023,0000,041-023,041-023-001,NaN,NaN,...,8000,800,80,COGHLAN,12,NORTE,2011,ENSENANZA,NaN,NaN
539433,-58.4728,-34.5641,398182,41,023,0000,041-023,041-023-001,NaN,NaN,...,8511,851,85,COGHLAN,12,NORTE,2011,SERVICOS SOCIALES Y DE SALUD,SERVICIOS DE ATENCION MEDICA,CENTROS MEDICOS Y DE ATENCION
539434,-58.4711,-34.5644,398183,41,023,0000,041-023,041-023-001,NaN,NaN,...,0,0,0,COGHLAN,12,NORTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN
539435,-58.4708,-34.5653,398184,41,023,0000,041-023,041-023-001,NaN,NaN,...,5238,523,52,COGHLAN,12,NORTE,2011,COMERCIO,VENTA AL POR MENOR ESPECIALIZADO NCP,VENTA DE DIARIOS Y REVISTAS
539436,-58.4712,-34.5655,398185,41,023,0000,041-023,041-023-001,NaN,NaN,...,5239,523,52,COGHLAN,12,NORTE,2011,COMERCIO,VENTA AL POR MENOR ESPECIALIZADO NCP,FLORERIA Y VIVEROS


Corrijo smp del RUS.

In [441]:
def parse_smp_id(smp_id):
    smp_id = unicode(smp_id)
    pattern = "([0-9]{1,5})([A-Z]{0,3})"
    try:
        match = re.match(pattern, smp_id, re.IGNORECASE).groups()
        return unicode(match[0]).lstrip("0").zfill(3) + match[1]
    except AttributeError:
        return smp_id

def correct_smp(seccion, manzana, parcela):
    secc = parse_smp_id(seccion)
    man = parse_smp_id(manzana)
    par = parse_smp_id(parcela)
    return "-".join([secc, man, par])
    
df.smp = map(correct_smp, df.seccion, df.manzana, df.parcela)

In [442]:
df_clasif.head()

,rama1,count,clasificacion
0,EDIFICIOS,284668,RESIDENCIAL
1,GARAGE PRIVADO,112170,RESIDENCIAL
2,COMERCIO,56608,COMERCIAL
3,LOCAL CERRADO,28805,OTROS
4,SERVICIOS COMPLEMENTARIOS AL TRANSPORTE,10310,SERVICIOS


In [443]:
serie_clasif = df_clasif.set_index("rama1")["clasificacion"]
serie_clasif.head()

rama1
EDIFICIOS                                  RESIDENCIAL
GARAGE PRIVADO                             RESIDENCIAL
COMERCIO                                     COMERCIAL
LOCAL CERRADO                                    OTROS
SERVICIOS COMPLEMENTARIOS AL TRANSPORTE      SERVICIOS
Name: clasificacion, dtype: object

In [444]:
print "Sólo", len(df[pd.isnull(df["rama1"])]), "parcelas no tienen un valor de rama1."

Sólo 7 parcelas no tienen un valor de rama1.


In [445]:
df = df.join(serie_clasif, on="rama1")
df.head()

,x,y,id,seccion,manzana,parcela,sm,smp,unificacio,parcelas_u,...,d31,d21,barrio,comuna,zona,ano,rama1,sub_rama1,ss_rama1,clasificacion
0,-58.4936,-34.5737,562177,63,066,020B,063-066,063-066-020B,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL
1,-58.4898,-34.5709,562178,63,116,001G,063-116,063-116-001G,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL
2,-58.4892,-34.5714,562179,63,116,006B,063-116,063-116-006B,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,EDIFICIOS,VIVIENDA,NaN,RESIDENCIAL
3,-58.4927,-34.5732,562180,63,079,022,063-079,063-079-022,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL
4,-58.4935,-34.5720,562181,63,080,029,063-080,063-080-029,NaN,NaN,...,0,0,VILLA URQUIZA,12,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL


* **Calcular el AREA de cada parcela** 
    * `ALTER TABLE parcelas ADD area float;`
    * `UPDATE parcelas SET area = ST_Area(geom);`

* **Asignar el radio que contiene a la parcela** 
    * `ALTER TABLE parcelas ADD CO_FRAC_RA text;`
    * `UPDATE parcelas SET CO_FRAC_RA = r.CO_FRAC_RA FROM radios_censo_2010 r WHERE ST_Contains(r.geom, parcelas.geom);`

* **Asignar el radio que intersecta a las parcelas que no están completamente contenidas**
    * `UPDATE parcelas SET CO_FRAC_RA = r.CO_FRAC_RA FROM radios_censo_2010 r WHERE NOT ST_Contains(r.geom, parcelas.geom) AND ST_Intersects(r.geom, parcelas.geom);`

* **Calcular el área intersectada por cada parcela con su radio (esto da un área incompleta)**
    * `ALTER TABLE parcelas ADD area_intersect float;`
    * `UPDATE parcelas SET area_intersect = ST_Area(ST_Intersection(r.geom, parcelas.geom)) FROM radios_censo_2010 r;`
        * Tarda demasiado, al parecer

* **Convertir tabla PostGIS en shp:** `pgsql2shp -f parcelas -h localhost -*p 5432 abenassi "SELECT * FROM parcelas;"`

In [446]:
parcelas_dbf = Dbf5("shp/divisiones/parcelas_joined/parcelas.dbf")

In [447]:
parcelas_df = parcelas_dbf.to_dataframe()

In [448]:
parcelas_df[pd.notnull(parcelas_df["AREA_INTER"])].head()

,FEATID1,MANZANA,OBS,PARCELA,SECCION,SMP,WEBLINK,PARC_ESQ,ROT_SEC,ROT_PAR,SM,PARTIDA,AREA,CO_FRAC_RA,AREA_INTER
9,380,074,INFORMACION CONSISTENTE,000B,001,001-074-000B,001-074-000B,S,01,NaN,001-074,0,12660.630585,1_12_11,8912.160755
10,379,074,INFORMACION CONSISTENTE,000A,001,001-074-000A,001-074-000A,S,01,NaN,001-074,0,12988.788728,1_12_11,10078.665520
2544,97,066B,INFORMACION CONSISTENTE,001a,003,003-066B-001a,003-066B-001a,S,03,NaN,003-066B,0,12270.938034,1_2_11,3709.955453
2545,96,066B,INFORMACION CONSISTENTE,001b,003,003-066B-001b,003-066B-001b,N,03,NaN,003-066B,0,2255.821207,1_2_11,1756.551665
2546,95,066B,INFORMACION CONSISTENTE,001c,003,003-066B-001c,003-066B-001c,N,03,NaN,003-066B,0,1299.047379,1_2_4,140.578700


In [449]:
parcelas_df["SMP"] = map(correct_smp, parcelas_df.SECCION, parcelas_df.MANZANA, parcelas_df.PARCELA)

In [450]:
parcelas_df_big = parcelas_df[pd.notnull(parcelas_df["AREA_INTER"])]

In [451]:
parcelas_df["area_ignorada"] = parcelas_df_big.AREA - parcelas_df_big.AREA_INTER

In [452]:
print len(parcelas_df[pd.notnull(parcelas_df["AREA_INTER"])]), "parcelas no están contenidas por un radio."

778 parcelas no están contenidas por un radio.


In [453]:
print len(parcelas_df[pd.isnull(parcelas_df["CO_FRAC_RA"])]), "no tienen radio asignado."

0 no tienen radio asignado.


In [454]:
print len(parcelas_df.CO_FRAC_RA.unique()), "radios con algún valor."

3483 radios con algún valor.


In [455]:
def select_area(area_inter, area):
    if pd.notnull(area_inter) and area_inter < area:
        return area_inter
    else:
        return area

parcelas_df["area_calculo"] = map(select_area, parcelas_df.AREA_INTER, parcelas_df.AREA)

In [456]:
print "Area total parcelas:", parcelas_df.AREA.sum() / 1000000
print "Area total radios:", df_radio.AREA.sum() / 1000000
print "Area parcelas ignorada:", parcelas_df["area_ignorada"].sum() / 1000000

Area total parcelas: 149.982080651
Area total radios: 200.428744499
Area parcelas ignorada: 9.86954063565


In [457]:
df_radio = get_or_create_indicators_df("RADIO")

In [458]:
# comparar sumatoria de áreas de parcelas con las áreas de los radios (no pueden ser mayores)

In [459]:
print df.columns[7]
cols = list(df.columns)
cols[7] = cols[7].upper()
df.columns = cols
df.columns

smp


Index([            u'x',             u'y',            u'id',       u'seccion',
             u'manzana',       u'parcela',            u'sm',           u'SMP',
          u'unificacio',    u'parcelas_u',       u'calle_1',           u'num',
             u'calle_2',        u'tipo_1',        u'tipo_2',         u'pisos',
              u'nombre',           u'd51',           u'd41',           u'd31',
                 u'd21',        u'barrio',        u'comuna',          u'zona',
                 u'ano',         u'rama1',     u'sub_rama1',      u'ss_rama1',
       u'clasificacion'],
      dtype='object')

In [460]:
df["SMP"] = df["SMP"].str.upper()
parcelas_df["SMP"] = parcelas_df["SMP"].str.upper()

In [461]:
len(df["SMP"].unique()), len(parcelas_df["SMP"].unique())

(319483, 318807)

In [462]:
df = df.join(parcelas_df[["SMP", "AREA", "AREA_INTER", "area_calculo", "CO_FRAC_RA"]].set_index("SMP"), on="SMP")
df.head()

,x,y,id,seccion,manzana,parcela,sm,SMP,unificacio,parcelas_u,...,zona,ano,rama1,sub_rama1,ss_rama1,clasificacion,AREA,AREA_INTER,area_calculo,CO_FRAC_RA
0,-58.4936,-34.5737,562177,63,066,020B,063-066,063-066-020B,NaN,NaN,...,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,251.412082,NaN,251.412082,12_16_1
1,-58.4898,-34.5709,562178,63,116,001G,063-116,063-116-001G,NaN,NaN,...,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,129.608734,NaN,129.608734,12_16_3
2,-58.4892,-34.5714,562179,63,116,006B,063-116,063-116-006B,NaN,NaN,...,OESTE,2011,EDIFICIOS,VIVIENDA,NaN,RESIDENCIAL,210.973922,NaN,210.973922,12_16_3
3,-58.4927,-34.5732,562180,63,079,022,063-079,063-079-022,NaN,NaN,...,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,278.004430,NaN,278.004430,12_16_1
4,-58.4935,-34.5720,562181,63,080,029,063-080,063-080-029,NaN,NaN,...,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,421.064637,NaN,421.064637,12_16_1


In [463]:
print len(df["SMP"].unique()), "parcelas con RUS."
print len(df[pd.isnull(df["CO_FRAC_RA"])]["SMP"].unique()), "parcelas sin radio asignado."
print len(df[pd.isnull(df["AREA"])]["SMP"].unique()), "parcelas sin area calculada."
print len(df[pd.isnull(df["area_calculo"])]["SMP"].unique()), "parcelas sin area calculada o intersección."
print len(df[pd.notnull(df["area_calculo"])]["SMP"].unique()), "parcelas con area calculada o intersección."

319483 parcelas con RUS.
6437 parcelas sin radio asignado.
6437 parcelas sin area calculada.
6437 parcelas sin area calculada o intersección.
313046 parcelas con area calculada o intersección.


In [464]:
def assign_radio_to_splitted_manzanas(id_radio, smp, parcelas_df):
    if pd.isnull(id_radio):
        s, m, p = smp.split("-")
        p = p[:3]
        smp = "-".join([s, m, p])

        radios = parcelas_df[parcelas_df.SMP.str.startswith(smp)]["CO_FRAC_RA"].unique()
        if len(radios) == 1:
            return radios[0]
    
    return id_radio

def aggregate_area_of_splitted_manzanas(id_radio, smp, area, parcelas_df):
    if pd.isnull(area) and pd.notnull(id_radio):
        s, m, p = smp.split("-")
        p = p[:3]
        smp = "-".join([s, m, p])
        return parcelas_df[parcelas_df.SMP.str.startswith(smp)]["AREA"].sum()

    return area

assign_radio_partial = partial(assign_radio_to_splitted_manzanas, parcelas_df=parcelas_df)
aggregate_area_partial = partial(aggregate_area_of_splitted_manzanas, parcelas_df=parcelas_df)

df.CO_FRAC_RA = map(assign_radio_partial, df.CO_FRAC_RA, df.SMP)
df.AREA = map(aggregate_area_partial, df.CO_FRAC_RA, df.SMP, df.AREA)
df["area_calculo"] = map(select_area, df.AREA_INTER, df.AREA)

In [465]:
print len(df["SMP"].unique()), "parcelas con RUS."
print len(df[pd.isnull(df["CO_FRAC_RA"])]["SMP"].unique()), "parcelas sin radio asignado."
print len(df[pd.isnull(df["AREA"])]["SMP"].unique()), "parcelas sin area calculada."
print len(df[pd.isnull(df["area_calculo"])]["SMP"].unique()), "parcelas sin area calculada o intersección."
print len(df[pd.notnull(df["area_calculo"])]["SMP"].unique()), "parcelas con area calculada o intersección."

319483 parcelas con RUS.
3953 parcelas sin radio asignado.
3953 parcelas sin area calculada.
3953 parcelas sin area calculada o intersección.
315530 parcelas con area calculada o intersección.


In [466]:
df[pd.isnull(df["CO_FRAC_RA"])]

,x,y,id,seccion,manzana,parcela,sm,SMP,unificacio,parcelas_u,...,zona,ano,rama1,sub_rama1,ss_rama1,clasificacion,AREA,AREA_INTER,area_calculo,CO_FRAC_RA
67,-58.5030,-34.5649,562250,55,015,005,055-015,055-015-005,NaN,NaN,...,NORTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,NaN,NaN,NaN,NaN
192,-58.4813,-34.6046,158,69,053B,000,069-053B,069-053B-000,NaN,NaN,...,OESTE,2011,EDIFICIOS,VIVIENDA,NaN,RESIDENCIAL,NaN,NaN,NaN,NaN
216,-58.5096,-34.5923,8119,83,151Z,plt1,083-151Z,083-151Z-PLT1,NaN,NaN,...,OESTE,2011,PLAZOLETA,PLAZOLETA,NaN,OTROS,NaN,NaN,NaN,NaN
218,-58.4890,-34.6014,8406,71,049Z,fsm1,071-049Z,071-049Z-FSM1,NaN,NaN,...,OESTE,2011,VIAS TREN,VIAS TREN,NaN,OTROS,NaN,NaN,NaN,NaN
3790,-58.4851,-34.6026,65242,69,046,023,069-046,069-046-023,U,022+023,...,OESTE,2011,UNIFICADO,UNIFICADO EDIFICIO,NaN,OTROS,NaN,NaN,NaN,NaN
3922,-58.4847,-34.6005,65377,69,049,032,069-049,069-049-032,NaN,NaN,...,OESTE,2011,EDIFICIOS,CASA,NaN,RESIDENCIAL,NaN,NaN,NaN,NaN
3923,-58.4847,-34.6005,65378,69,049,032,069-049,069-049-032,NaN,NaN,...,OESTE,2011,GARAGE PRIVADO,GARAGE PRIVADO,NaN,RESIDENCIAL,NaN,NaN,NaN,NaN
3934,-58.4867,-34.6012,65389,69,049Z,fsm1,069-049Z,069-049Z-FSM1,NaN,NaN,...,OESTE,2011,VIAS TREN,VIAS TREN,NaN,OTROS,NaN,NaN,NaN,NaN
4395,-58.4827,-34.5982,65854,69,062Z,plt1,069-062Z,069-062Z-PLT1,NaN,NaN,...,OESTE,2011,PLAZOLETA,PLAZOLETA,NaN,OTROS,NaN,NaN,NaN,NaN
4604,-58.4829,-34.5992,66068,69,064,028,069-064,069-064-028,U,027B + 028 + 029,...,OESTE,2011,UNIFICADO,UNIFICADO LOTE,NaN,OTROS,NaN,NaN,NaN,NaN


In [469]:
parcelas_df[parcelas_df.SMP.str.startswith("069-053B")]

,FEATID1,MANZANA,OBS,PARCELA,SECCION,SMP,WEBLINK,PARC_ESQ,ROT_SEC,ROT_PAR,SM,PARTIDA,AREA,CO_FRAC_RA,AREA_INTER,area_ignorada,area_calculo
243782,1283,053B,INFORMACION CONSISTENTE,001,069,069-053B-001,069-053B-001,S,69,NaN,069-053B,252888,226.149333,11_11_2,NaN,NaN,226.149333
243815,1250,053B,INFORMACION CONSISTENTE,023,069,069-053B-023,069-053B-023,N,69,NaN,069-053B,252889,218.939481,11_11_2,NaN,NaN,218.939481
244194,871,053B,INFORMACION CONSISTENTE,022,069,069-053B-022,069-053B-022,N,69,NaN,069-053B,252890,225.119492,11_11_2,NaN,NaN,225.119492
244240,505,053B,INFORMACION CONSISTENTE,015a,069,069-053B-015A,069-053B-015a,N,69,NaN,069-053B,0,428.322686,11_11_2,NaN,NaN,428.322686
244262,483,053B,INFORMACION CONSISTENTE,014a,069,069-053B-014A,069-053B-014a,N,69,NaN,069-053B,0,171.134410,11_11_2,NaN,NaN,171.134410
244268,477,053B,INFORMACION CONSISTENTE,005l,069,069-053B-005L,069-053B-005l,N,69,NaN,069-053B,0,437.238062,11_11_2,NaN,NaN,437.238062
244288,457,053B,INFORMACION CONSISTENTE,013,069,069-053B-013,069-053B-013,N,69,NaN,069-053B,252899,690.457046,11_11_2,NaN,NaN,690.457046
244694,531,053B,INFORMACION CONSISTENTE,016,069,069-053B-016,069-053B-016,N,69,NaN,069-053B,252896,148.931245,11_11_2,NaN,NaN,148.931245
244705,520,053B,INFORMACION CONSISTENTE,005h,069,069-053B-005H,069-053B-005h,N,69,NaN,069-053B,0,387.883164,11_11_2,NaN,NaN,387.883164
244711,834,053B,INFORMACION CONSISTENTE,002,069,069-053B-002,069-053B-002,N,69,NaN,069-053B,252887,248.574233,11_11_2,NaN,NaN,248.574233


In [478]:
len(df), len(df.SMP.unique())

(555756, 319483)

In [480]:
smp_count = df.groupby(by="SMP").count()["area_calculo"].to_dict()

In [481]:
df["smp_count"] = df.SMP.map(smp_count)

In [482]:
df["AREA_POR_USO"] = df.area_calculo / df.smp_count

In [484]:
df[df["CO_FRAC_RA"] == "12_16_1"].groupby(by="clasificacion").sum()["area_calculo"]

clasificacion
COMERCIAL       3812.530654
OTROS           5292.368953
RESIDENCIAL    90434.046597
SERVICIOS       5381.848152
Name: area_calculo, dtype: float64

In [485]:
uso_por_radio = df.groupby(by=["CO_FRAC_RA", "clasificacion"]).sum()["area_calculo"] / df.groupby(by="CO_FRAC_RA").sum()["area_calculo"]
uso_por_radio[:20]

CO_FRAC_RA  clasificacion
10_10_1     COMERCIAL        0.038427
            OTROS            0.036113
            RESIDENCIAL      0.846775
            SERVICIOS        0.078685
10_10_10    COMERCIAL        0.045252
            OTROS            0.028980
            RESIDENCIAL      0.891986
            SERVICIOS        0.033782
10_10_11    COMERCIAL        0.016801
            INDUSTRIAL       0.004113
            OTROS            0.030662
            RESIDENCIAL      0.903959
            SERVICIOS        0.044465
10_10_12    COMERCIAL        0.050873
            INDUSTRIAL       0.008872
            OTROS            0.087954
            RESIDENCIAL      0.767932
            SERVICIOS        0.084369
10_10_2     COMERCIAL        0.019215
            INDUSTRIAL       0.005497
Name: area_calculo, dtype: float64

In [486]:
print len(uso_por_radio.groupby(level=0)), "radios tienen el uso del suelo calculado"

3465 radios tienen el uso del suelo calculado


In [487]:
from scripts.create_indicators import get_or_create_indicators_df

indicators_radio = get_or_create_indicators_df("RADIO")

In [488]:
def get_land_use(radio, use, serie):
    if not radio in serie:
        return pd.np.nan
    
    if use in serie[radio]:
        return serie[radio][use]
    else:
        return 0

In [489]:
get_residencial_use = partial(get_land_use, use="RESIDENCIAL", serie=uso_por_radio)
get_comercial_use = partial(get_land_use, use="COMERCIAL", serie=uso_por_radio)
get_industrial_use = partial(get_land_use, use="INDUSTRIAL", serie=uso_por_radio)
get_servicios_use = partial(get_land_use, use="SERVICIOS", serie=uso_por_radio)
get_otros_use = partial(get_land_use, use="OTROS", serie=uso_por_radio)

In [490]:
indicators_radio["RESIDENCIAL"] = indicators_radio["CO_FRAC_RA"].map(get_residencial_use)
indicators_radio["COMERCIAL"] = indicators_radio["CO_FRAC_RA"].map(get_comercial_use)
indicators_radio["INDUSTRIAL"] = indicators_radio["CO_FRAC_RA"].map(get_industrial_use)
indicators_radio["SERVICIOS"] = indicators_radio["CO_FRAC_RA"].map(get_servicios_use)
indicators_radio["OTROS"] = indicators_radio["CO_FRAC_RA"].map(get_otros_use)

In [491]:
indicators_radio.tail()

,empleo,desocup,inact,ocup_viv,0_14,15_64,mas_65,hab,con_satisf,con_basica,...,educ_priv,educ_pub,hospitales,esp_verde,nse_alt,RESIDENCIAL,CO_FRAC_RA,AREA,area_km2,hab_km2
Código,,,,,,,,,,,,,,,,,,,,,
020152109,0.740891,0.035149,0.238866,0.746063,0.141367,0.698725,0.159907,863,0.796834,0.203166,...,0,0,0,0,3,0.753259,15_21_9,19797.823914,0.019798,43590.649345
020152110,0.716763,0.031250,0.267823,0.708010,0.153344,0.698206,0.148450,613,0.868613,0.062044,...,0,0,0,0,3,0.819678,15_21_10,16326.588379,0.016327,37546.117154
020152111,0.811037,0.030000,0.168896,0.739726,0.142037,0.736011,0.121951,697,0.956790,0.030864,...,0,0,0,0,3,0.833505,15_21_11,17678.281403,0.017678,39426.909445
020152112,0.760820,0.037464,0.216401,0.778986,0.139216,0.735294,0.125490,510,0.860465,0.074419,...,0,0,0,0,3,0.748979,15_21_12,20275.417023,0.020275,25153.613335
020152113,0.691517,0.046099,0.244216,0.770161,0.130726,0.689385,0.179888,895,0.869110,0.099476,...,0,0,0,0,3,0.535947,15_21_13,38614.865514,0.038615,23177.602410


In [492]:
indicators_radio.to_csv(get_indicators_path("RADIO"), encoding="utf-8")